In [97]:
import pandas as pd
import numpy as np
from collections import defaultdict
import re
from collections import Counter
import math

In [6]:
test = pd.read_csv("Questions.csv",  encoding='latin-1')

In [12]:
test.head()

,Id,OwnerUserId,CreationDate,Score,Title,Body
0,469,147.0,2008-08-02T15:11:16Z,21,How can I find the full path to a font from it...,<p>I am using the Photoshop's javascript API t...
1,502,147.0,2008-08-02T17:01:58Z,27,Get a preview JPEG of a PDF on Windows?,<p>I have a cross-platform (Python) applicatio...
2,535,154.0,2008-08-02T18:43:54Z,40,Continuous Integration System for a Python Cod...,<p>I'm starting work on a hobby project with a...
3,594,116.0,2008-08-03T01:15:08Z,25,cx_Oracle: How do I iterate over a result set?,<p>There are several ways to iterate over a re...
4,683,199.0,2008-08-03T13:19:16Z,28,Using 'in' to match an attribute of Python obj...,<p>I don't remember whether I was dreaming or ...


In [231]:
small_test = test['Title'].head(500)

In [232]:
def coalesce(value):
    if value == 0:
        return np.NaN
    else:
        return value

In [233]:
all_unique_words = []
for document in small_test:
    for word in re.sub('[^A-Za-z0-9]+', ' ', document).split():
        if word.lower() not in all_unique_words:
            all_unique_words.append(word.lower())

In [234]:
all_words = []
for document in small_test:
    for word in document.split():
        all_words.append(word.lower())
counts = Counter(all_words)

In [237]:
tf = defaultdict(list)
for i in all_unique_words:
    for index, document in enumerate(small_test):
        frequency = len(re.findall(i.lower(),document.lower()))
        overall_frequency = counts[i]
        tf[i].append(frequency/coalesce(overall_frequency))

In [238]:
total_documents = len(small_test)

In [239]:
documents_with_word = defaultdict(int)

for document in small_test:
    for word in all_unique_words:
        if word in document.lower():
            documents_with_word[word] += 1
            

In [240]:
tf_idf = defaultdict(int)

for word in all_unique_words:
    tf_idf[word] = pd.Series(tf[word]) * math.log(total_documents/coalesce(documents_with_word[word]))

In [241]:
def most_relevant_doc(word):
    ## for a given word return the most relevant document ##
    index_max = np.argmax(tf_idf[word])
    return [small_test[np.argmax(tf_idf[word])], word, tf_idf[word].max()]
    
    

In [247]:
most_relevant_doc("words")

/Users/shirleyliu/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:52: FutureWarning: 'argmax' is deprecated. Use 'idxmax' instead. The behavior of 'argmax' will be corrected to return the positional maximum in the future. Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


['List all words in a dictionary that start with <user input>',
 'words',
 6.214608098422191]